# **`Project 3: Team John Hicks`**

## **Mali** File

### **Objective**: Compare the nutritional intake and food intake in America vs. Mali before, during, and after the pandemic.

#### Group Members:
> Leona Katibah <br>
> Jack Pollard <br>
> Jakob Mackanych <br>
> Xi Wang <br>
> Wendy Peng <br>
> Sury Dongre

# Project Setup

In [1]:
#import all packages here
!pip install -r requirements.txt
!pip install eep153_tools python_gnupg gspread_pandas cfe --quiet

In [8]:
import pandas as pd
import numpy as np
from eep153_tools.sheets import read_sheets

In [9]:
Mali_Data = '1FnbKj5gwoAQb0QSunBQmyExHwaiLEAdAFCxL59SWLyg'

## **2014-2015 Mali Data**

In [10]:
# Change 'Mali_Data' to key of your own sheet in Sheets, above
x = read_sheets(Mali_Data,sheet='Food Expenditures (2014-15)')
x = x.set_index(['i','t','m','j']).squeeze()


# Now prices
p = read_sheets(Mali_Data,sheet='Food Prices (2014-15)').set_index(['t','m','j','u'])

# Compute medians of prices for particular time, place and unit
p = p.groupby(['t','m','j','u']).median()

# Just keep metric units
p = p.xs('kg',level="u").squeeze().unstack('j')

# Drop prices for goods we don't have expenditures for
p = p[x.index.levels[-1]].T

# Household characteristics
d = read_sheets(Mali_Data,sheet="Household Characteristics")
#sort data to only have 2014-15 Household charateristics
d = d[d.t == '2014-15']
d.columns.name = 'k'

# Fill blanks with zeros
d = d.replace(np.nan,0)

# Expenditures x may have duplicate columns
x = x.T.groupby(['i','t','m','j']).sum()
x = x.replace(0,np.nan) # Replace zeros with missing

# Take logs of expenditures; call this y
y = np.log(x)

d.set_index(['i','t','m'],inplace=True)


x.head()
d.head()
y.head()

i     t        m      j                         
1003  2014-15  Kayes  Feuilles de Haricot           5.703782
                      Feuilles de baobab            5.298317
                      Thé en paquet ou en sachet    6.551080
1006  2014-15  Kayes  Poisson fumé                  8.160518
                      Poisson séché                 7.313220
Name: expenditure, dtype: float64

## **2021-22 Mali Data**

In [11]:
# Change 'Mali_Data' to key of your own sheet in Sheets, above
x_2 = read_sheets(Mali_Data,sheet='Food Expenditures (2021-22)')
x_2 = x_2.set_index(['i','t','m','j']).squeeze()


# Now prices
p_2 = read_sheets(Mali_Data,sheet='Food Prices (2021-22)').set_index(['t','m','j','u'])

# Compute medians of prices for particular time, place and unit
p_2 = p_2.groupby(['t','m','j','u']).median()

# Just keep metric units
p_2 = p_2.xs('kg',level="u").squeeze().unstack('j')

# Drop prices for goods we don't have expenditures for
p = p_2[x_2.index.levels[-1]].T

# Household characteristics
d_2 = read_sheets(Mali_Data,sheet="Household Characteristics")
#sort data to only have 2021-22 Household charateristics
d_2 = d_2[d_2.t == '2021-22']
d_2.columns.name = 'k'

# Fill blanks with zeros
d_2 = d_2.replace(np.nan,0)

# Expenditures x may have duplicate columns
x_2 = x_2.T.groupby(['i','t','m','j']).sum()
x_2 = x_2.replace(0,np.nan) # Replace zeros with missing

# Take logs of expenditures; call this y
y_2 = np.log(x_2)

d_2.set_index(['i','t','m'],inplace=True)

x_2.head()
d_2.head()
y_2.head()

i     t        m      j             
1001  2021-22  Kayes  Banane douce      7.090077
                      Carpe             6.684612
                      Lait en poudre    6.551080
                      Mayonnaise        4.317488
                      Moutarde          3.218876
Name: expenditure, dtype: float64

# **[A] Choice of a Population -  with supporting expenditure data**

- **Population chosen:** Households in **Mali** and in USA - but this document focuses only on the Mali data to maintain clarity across datasets.

- **Time periods:** Using data from **2014–15** (pre-COVID) and **2021–22** (post-COVID)
- **Reason for choice:** To explore how the COVID-19 pandemic impacted *food demand and consumption patterns*
- **Data available:** Detailed household-level data on *food expenditures, prices, and demographics*



### Function to Dynamically Select a **Year** (2014–15 or 2021–22)


In [12]:
def process_data_per_year(mali_data, year):
    """
    Processes the Mali dataset for a given year.
    
    Parameters:
      mali_data: The key or identifier for your Sheets data.
      year_str: The time period to filter by (e.g., "2021-22").
    
    Returns:
      x: DataFrame of processed food expenditures.
      p: DataFrame of processed food prices (only for goods with expenditures).
      y: DataFrame of the log of expenditures.
      d: DataFrame of household characteristics for the selected year.
    """
    # --- Food Expenditures ---
    sheet_expend = f"Food Expenditures ({year})"
    x = read_sheets(mali_data, sheet=sheet_expend)
    # Set multi-index and squeeze if possible (your code expects a Series)
    x = x.set_index(['i','t','m','j']).squeeze()
    
    # --- Food Prices ---
    sheet_prices = f"Food Prices ({year})"
    p = read_sheets(mali_data, sheet=sheet_prices).set_index(['t','m','j','u'])
    # Compute medians of prices for each time, place, good, and unit
    p = p.groupby(['t','m','j','u']).median()
    # Keep only metric units (e.g., 'kg')
    p = p.xs('kg', level="u").squeeze().unstack('j')
    # Drop prices for goods we don't have expenditures for
    p = p[x.index.levels[-1]].T

    # --- Household Characteristics ---
    d = read_sheets(mali_data, sheet="Household Characteristics")
    # Filter the data for the selected year
    d = d[d.t == year]
    # Set column names attribute and replace missing values with zeros
    d.columns.name = 'k'
    d = d.replace(np.nan, 0)
    
    # --- Expenditures Processing ---
    x = x.T.groupby(['i','t','m','j']).sum()
    x = x.replace(0, np.nan)
    # Take logarithms of expenditures; call this y
    y = np.log(x)
    
    # --- Set index for household characteristics ---
    d.set_index(['i','t','m'], inplace=True)
    
    return x, y, d

#Change Year and Run Cell
x, y, d = process_data_per_year(Mali_Data, "2014-15")

print("Food Expenditures (x)")
display(x.head())

print("Household Characteristics (d)")
display(d.head())

print("Log of Expenditures (y)")
display(y.head())

#print("Prices (p)")
#display(p.head())

Food Expenditures (x)


i     t        m      j                         
1003  2014-15  Kayes  Feuilles de Haricot            300
                      Feuilles de baobab             200
                      Thé en paquet ou en sachet     700
1006  2014-15  Kayes  Poisson fumé                  3500
                      Poisson séché                 1500
Name: expenditure, dtype: int64

Household Characteristics (d)


,,k,F 00-03,F 04-08,F 09-13,F 14-18,F 19-30,F 31-50,F 51+,M 00-03,M 04-08,M 09-13,M 14-18,M 19-30,M 31-50,M 51+,M NA,log HSize,F NA
i,t,m,,,,,,,,,,,,,,,,,
1003,2014-15,Kayes,0.0,1.0,0.0,0.0,2.0,0.0,0.0,4.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0,2.197225,0.0
1006,2014-15,Kayes,1.0,2.0,1.0,1.0,2.0,1.0,1.0,1.0,0.0,2.0,2.0,0.0,0.0,1.0,0.0,2.708050,0.0
1009,2014-15,Kayes,4.0,2.0,2.0,1.0,7.0,2.0,1.0,1.0,3.0,3.0,1.0,2.0,5.0,0.0,0.0,3.526361,0.0
10003,2014-15,Kayes,1.0,0.0,1.0,3.0,1.0,1.0,1.0,0.0,1.0,2.0,0.0,2.0,0.0,1.0,0.0,2.639057,0.0
10006,2014-15,Kayes,2.0,0.0,0.0,2.0,2.0,1.0,0.0,0.0,1.0,0.0,1.0,3.0,0.0,0.0,0.0,2.484907,0.0


Log of Expenditures (y)


i     t        m      j                         
1003  2014-15  Kayes  Feuilles de Haricot           5.703782
                      Feuilles de baobab            5.298317
                      Thé en paquet ou en sachet    6.551080
1006  2014-15  Kayes  Poisson fumé                  8.160518
                      Poisson séché                 7.313220
Name: expenditure, dtype: float64

# **[B] Nutritional content of differnet foods**

In [4]:
sheet_id = '1FnbKj5gwoAQb0QSunBQmyExHwaiLEAdAFCxL59SWLyg'

InputFiles = {
    'Expenditures_2014-15': (sheet_id, 'Food Expenditures (2014-15)'),
    'Expenditures_2021-22': (sheet_id, 'Food Expenditures (2021-22)'),
    'Prices_2014-15':       (sheet_id, 'Food Prices (2014-15)'),
    'Prices_2021-22':       (sheet_id, 'Food Prices (2021-22)'),
    'HH_Characteristics':   (sheet_id, 'Household Characteristics'),
    'FCT':('1yVLriVpo7KGUXvR3hq_n53XpXlD5NmLaH1oOMZyV0gQ','FCT'),
    'RDI':('1yVLriVpo7KGUXvR3hq_n53XpXlD5NmLaH1oOMZyV0gQ','RDI')
            
}

### Function to Dynamically Select a **Year** (2014–15 or 2021–22) and clean DFs

In [5]:
def get_clean_sheet(key, sheet=None):
    df = read_sheets(key, sheet=sheet)
    df.columns = [c.strip() for c in df.columns]
    df = df.loc[:, ~df.columns.duplicated(keep='first')]
    df = df.drop([col for col in df.columns if col.startswith('Unnamed')], axis=1)
    df = df.loc[~df.index.duplicated(), :]
    return df

def load_data_by_year(year):
    """
    Dynamically loads and cleans data for a given year (e.g., '2014-15' or '2021-22').
    Returns x_2 (expenditures), z_2 (household characteristics),
    p_2 (prices), fct (food composition), rdi (recommended dietary intake).
    """

    # 1) Expenditures
    x_raw = get_clean_sheet(
        InputFiles[f'Expenditures_{year}'][0],
        sheet=InputFiles[f'Expenditures_{year}'][1]
    )
    if 'm' not in x_raw.columns:
        x_raw['m'] = 1
    x_raw.columns = [col.strip() for col in x_raw.columns]
    x_2 = x_raw.pivot_table(index=['i', 't', 'm'], columns='j', values='expenditure', aggfunc='sum')
    x_2.columns.name = 'j'
    x_2 = x_2.apply(pd.to_numeric, errors='coerce').replace(0, np.nan)

    # 2) Household Characteristics (same key for all years)
    z_2 = get_clean_sheet(
        InputFiles['HH_Characteristics'][0],
        sheet=InputFiles['HH_Characteristics'][1]
    )
    if 'm' not in z_2.columns:
        z_2['m'] = 1
    z_2 = z_2.set_index(['i', 't', 'm'])
    z_2.columns.name = 'k'
    z_2 = z_2.apply(pd.to_numeric, errors='coerce')

    # 3) Prices
    p_2 = get_clean_sheet(
        InputFiles[f'Prices_{year}'][0],
        sheet=InputFiles[f'Prices_{year}'][1]
    )
    if 'm' not in p_2.columns:
        p_2['m'] = 1
    p_2 = p_2.set_index(['t', 'm'])
    p_2.columns.name = 'j'
    p_2 = p_2.apply(pd.to_numeric, errors='coerce').replace(0, np.nan)

    # 4) Food Composition Table (FCT) - not year-based
    fct = get_clean_sheet(
        InputFiles['FCT'][0],
        sheet=InputFiles['FCT'][1]
    )
    fct = fct.set_index('j')
    fct.columns.name = 'n'
    fct = fct.apply(pd.to_numeric, errors='coerce')

    # 5) Recommended Dietary Intake (RDI) - not year-based
    rdi = get_clean_sheet(
        InputFiles['RDI'][0],
        sheet=InputFiles['RDI'][1]
    )
    rdi = rdi.set_index('n')
    rdi.columns.name = 'k'

    return x_2, z_2, p_2, fct, rdi

# Load data
year = '2021-22'  # or '2014-15'
x_2, z_2, p_2, fct, rdi = load_data_by_year(year)

# Display output
print(f"Expenditures for {year}:")
display(x_2.head())

print("Household Characteristics (z_2):")
display(z_2.head())

print(f"Prices for {year}:")
display(p_2.head())

print("FCT:")
display(fct.head())

print("RDI:")
display(rdi.head())

# Shapes
print("Shape of x_2 (Expenditures):", x_2.shape)
print("Shape of z_2 (Household Characteristics):", z_2.shape)
print("Shape of p_2 (Prices):", p_2.shape)
print("Shape of fct (Food Composition Table):", fct.shape)
print("Shape of rdi (Recommended Daily Intake):", rdi.shape)


Expenditures for 2021-22:


,,j,Ail,Ananas,Arachide,Attiéke,Aubergine,Autres agrumes,Autres farines de céréales,Autres fruits,Autres légumes en feuilles,Autres produits alimentaires,...,Viande de bœuf,Viande de chameau,Viande de chèvre,Viande de mouton,Viande de porc,Viande de poulet,Viande séchée,Vinaigre,Vinaigre de citron,Œufs
i,t,m,,,,,,,,,,,,,,,,,,,,,
1001,2021-22,Kayes,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,1000.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1002,2021-22,Kayes,NaN,NaN,NaN,NaN,500.0,NaN,NaN,NaN,NaN,NaN,...,1000.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1003,2021-22,Kayes,NaN,NaN,NaN,NaN,200.0,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1004,2021-22,Kayes,NaN,NaN,NaN,NaN,150.0,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1005,2021-22,Kayes,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,2000.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,375.0


Household Characteristics (z_2):


,,k,F 00-03,F 04-08,F 09-13,F 14-18,F 19-30,F 31-50,F 51+,M 00-03,M 04-08,M 09-13,M 14-18,M 19-30,M 31-50,M 51+,M NA,log HSize,F NA
i,t,m,,,,,,,,,,,,,,,,,
1001,2017-18,Kayes,NaN,2.0,1.0,NaN,2.0,NaN,NaN,1.0,2.0,NaN,NaN,NaN,1.0,NaN,NaN,2.197225,NaN
1002,2017-18,Kayes,3.0,NaN,NaN,2.0,2.0,1.0,NaN,NaN,1.0,NaN,NaN,1.0,1.0,1.0,NaN,2.484907,NaN
1003,2017-18,Kayes,NaN,3.0,4.0,4.0,1.0,2.0,1.0,2.0,2.0,2.0,1.0,1.0,NaN,NaN,NaN,3.135494,NaN
1004,2017-18,Kayes,6.0,3.0,5.0,6.0,5.0,4.0,1.0,2.0,6.0,3.0,1.0,3.0,3.0,1.0,NaN,3.891820,NaN
1005,2017-18,Kayes,1.0,3.0,1.0,4.0,2.0,3.0,2.0,NaN,2.0,1.0,2.0,1.0,NaN,1.0,NaN,3.135494,NaN


Prices for 2021-22:


...

FCT:


n,Energy,Protein,Fiber,Folate,Calcium,Carbohydrate,Iron,Niacin,Riboflavin,Thiamin,Vitamin A,Vitamin B-12,Vitamin B-6,Vitamin C,Zinc
j,,,,,,,,,,,,,,,
Avocado,1600,20.0,70.0,810,120,85.0,6.0,17.38,1.30,0.67,70,0.0,2.57,100,6.0
Beans (dry),1700,98.0,60.0,500,580,325.0,30.0,7.00,1.20,3.40,0,0.0,2.15,10,8.0
Beans (fresh),3470,214.0,160.0,5250,1130,626.0,51.0,11.74,2.12,7.13,0,0.0,4.74,63,23.0
Beef,2510,182.0,0.0,60,70,0.0,19.0,31.50,1.60,0.90,0,28.9,3.80,0,37.0
Beef (roasted),2910,264.0,0.0,70,90,0.0,27.0,37.20,2.20,0.90,0,24.7,3.40,0,60.0


RDI:


k,F 00-03,M 00-03,F 04-08,M 04-08,F 09-13,M 09-13,F 14-18,M 14-18,F 19-30,M 19-30,F 31-50,M 31-50,F 51+,M 51+
n,,,,,,,,,,,,,,
Energy,1000.0,1000.0,1200.0,1400.0,1600.0,1800.0,1800.0,2200.0,2000.0,2400.0,1800.0,2200.0,1600.0,2000.0
Protein,13.0,13.0,19.0,19.0,34.0,34.0,46.0,52.0,46.0,56.0,46.0,56.0,46.0,56.0
Fiber,14.0,14.0,16.8,19.6,22.4,25.2,25.2,30.8,28.0,33.6,25.2,30.8,22.4,28.0
Folate,150.0,150.0,200.0,200.0,300.0,300.0,400.0,400.0,400.0,400.0,400.0,400.0,400.0,400.0
Calcium,700.0,700.0,1000.0,1000.0,1300.0,1300.0,1300.0,1300.0,1000.0,1000.0,1000.0,1000.0,1200.0,1000.0


Shape of x_2 (Expenditures): (6125, 132)
Shape of z_2 (Household Characteristics): (24939, 17)
Shape of p_2 (Prices): (1007, 3)
Shape of fct (Food Composition Table): (103, 15)
Shape of rdi (Recommended Daily Intake): (20, 14)


In [6]:
print("y_2 type:", type(x_2.stack()))
print("z_2 type:", type(z_2))
print("y_2 index names:", x_2.stack().index.names)
print("z_2 index names:", z_2.index.names)

y_2 type: <class 'pandas.core.series.Series'>
z_2 type: <class 'pandas.core.frame.DataFrame'>
y_2 index names: ['i', 't', 'm', 'j']
z_2 index names: ['i', 't', 'm']


In [14]:
import cfe

result = cfe.Regression(y=np.log(x.stack()),d=z)

result.get_beta().sort_values(ascending=False) # Check sanity...

AttributeError: 'Series' object has no attribute 'stack'

#  [B] Nutritional adequacy of diet 

# [A] Estimate Demand System

# [C] Counterfactual Experiments

# extra old code

In [78]:
def get_clean_sheet(key, sheet=None):
    
    df = read_sheets(key, sheet=sheet)
    df.columns = [c.strip() for c in df.columns]
    df = df.loc[:, ~df.columns.duplicated(keep='first')]
    df = df.drop([col for col in df.columns if col.startswith('Unnamed')], axis=1)
    df = df.loc[~df.index.duplicated(), :]
    return df

def load_data_by_year(year):
    """
    Dynamically loads and cleans data for a given year (e.g., '2014-15' or '2021-22').
    Returns x (expenditures), z (household characteristics),
    p (prices), fct (food composition), rdi (recommended dietary intake).
    """

    # 1) Expenditures
    x = get_clean_sheet(
        InputFiles[f'Expenditures_{year}'][0],
        sheet=InputFiles[f'Expenditures_{year}'][1]
    )
    if 'm' not in x.columns:
        x['m'] = 1
    x = x.set_index(['i', 't', 'm'])
    x.columns.name = 'j'
    x = x.apply(pd.to_numeric, errors='coerce').replace(0, np.nan)

    # 2) Household Characteristics (same key for all years)
    z = get_clean_sheet(
        InputFiles['HH_Characteristics'][0],
        sheet=InputFiles['HH_Characteristics'][1]
    )
    if 'm' not in z.columns:
        z['m'] = 1
    z = z.set_index(['i', 't', 'm'])
    z.columns.name = 'k'
    z = z.apply(pd.to_numeric, errors='coerce')

    # 3) Prices
    p = get_clean_sheet(
        InputFiles[f'Prices_{year}'][0],
        sheet=InputFiles[f'Prices_{year}'][1]
    )
    if 'm' not in p.columns:
        p['m'] = 1
    p = p.set_index(['t', 'm'])
    p.columns.name = 'j'
    p = p.apply(pd.to_numeric, errors='coerce').replace(0, np.nan)

    # 4) Food Composition Table (FCT) - not year-based
    fct = get_clean_sheet(
        InputFiles['FCT'][0],
        sheet=InputFiles['FCT'][1]
    )
    fct = fct.set_index('j')
    fct.columns.name = 'n'
    fct = fct.apply(pd.to_numeric, errors='coerce')

    # 5) Recommended Dietary Intake (RDI) - not year-based
    rdi = get_clean_sheet(
        InputFiles['RDI'][0],
        sheet=InputFiles['RDI'][1]
    )
    rdi = rdi.set_index('n')
    rdi.columns.name = 'k'

    return x_2, z_2, p_2, fct, rdi

year = '2021-22'  # SELECT '2014-15' OR '2021-22'

x_2, z_2, p_2, fct, rdi = load_data_by_year(year)

print(f"Expenditures for {year}:")
display(x.head())

print("Household Characteristics (z):")
display(z.head())

print(f"Prices for {year}:")
display(p.head())

print("FCT:")
display(fct.head())

print("RDI:")
display(rdi.head())


print("Shape of x (Expenditures):", x.shape)
print("Shape of z (Household Characteristics):", z.shape)
print("Shape of p (Prices):", p.shape)
print("Shape of fct (Food Composition Table):", fct.shape)
print("Shape of rdi (Recommended Daily Intake):", rdi.shape)

Expenditures for 2021-22:


j                        j  expenditure
i       t       m                      
1015001 2021-22 Bamako NaN          600
                Bamako NaN           50
                Bamako NaN          300
                Bamako NaN          300
                Bamako NaN         1100

Household Characteristics (z):


,,k,F 00-03,F 04-08,F 09-13,F 14-18,F 19-30,F 31-50,F 51+,M 00-03,M 04-08,M 09-13,M 14-18,M 19-30,M 31-50,M 51+,M NA,log HSize,F NA
i,t,m,,,,,,,,,,,,,,,,,
1001,2017-18,Kayes,NaN,2.0,1.0,NaN,2.0,NaN,NaN,1.0,2.0,NaN,NaN,NaN,1.0,NaN,NaN,2.197225,NaN
1002,2017-18,Kayes,3.0,NaN,NaN,2.0,2.0,1.0,NaN,NaN,1.0,NaN,NaN,1.0,1.0,1.0,NaN,2.484907,NaN
1003,2017-18,Kayes,NaN,3.0,4.0,4.0,1.0,2.0,1.0,2.0,2.0,2.0,1.0,1.0,NaN,NaN,NaN,3.135494,NaN
1004,2017-18,Kayes,6.0,3.0,5.0,6.0,5.0,4.0,1.0,2.0,6.0,3.0,1.0,3.0,3.0,1.0,NaN,3.891820,NaN
1005,2017-18,Kayes,1.0,3.0,1.0,4.0,2.0,3.0,2.0,NaN,2.0,1.0,2.0,1.0,NaN,1.0,NaN,3.135494,NaN


Prices for 2021-22:


...

FCT:


n,Energy,Protein,Fiber,Folate,Calcium,Carbohydrate,Iron,Niacin,Riboflavin,Thiamin,Vitamin A,Vitamin B-12,Vitamin B-6,Vitamin C,Zinc
j,,,,,,,,,,,,,,,
Avocado,1600,20.0,70.0,810,120,85.0,6.0,17.38,1.30,0.67,70,0.0,2.57,100,6.0
Beans (dry),1700,98.0,60.0,500,580,325.0,30.0,7.00,1.20,3.40,0,0.0,2.15,10,8.0
Beans (fresh),3470,214.0,160.0,5250,1130,626.0,51.0,11.74,2.12,7.13,0,0.0,4.74,63,23.0
Beef,2510,182.0,0.0,60,70,0.0,19.0,31.50,1.60,0.90,0,28.9,3.80,0,37.0
Beef (roasted),2910,264.0,0.0,70,90,0.0,27.0,37.20,2.20,0.90,0,24.7,3.40,0,60.0


RDI:


k,F 00-03,M 00-03,F 04-08,M 04-08,F 09-13,M 09-13,F 14-18,M 14-18,F 19-30,M 19-30,F 31-50,M 31-50,F 51+,M 51+
n,,,,,,,,,,,,,,
Energy,1000.0,1000.0,1200.0,1400.0,1600.0,1800.0,1800.0,2200.0,2000.0,2400.0,1800.0,2200.0,1600.0,2000.0
Protein,13.0,13.0,19.0,19.0,34.0,34.0,46.0,52.0,46.0,56.0,46.0,56.0,46.0,56.0
Fiber,14.0,14.0,16.8,19.6,22.4,25.2,25.2,30.8,28.0,33.6,25.2,30.8,22.4,28.0
Folate,150.0,150.0,200.0,200.0,300.0,300.0,400.0,400.0,400.0,400.0,400.0,400.0,400.0,400.0
Calcium,700.0,700.0,1000.0,1000.0,1300.0,1300.0,1300.0,1300.0,1000.0,1000.0,1000.0,1000.0,1200.0,1000.0


Shape of x (Expenditures): (112505, 2)
Shape of z (Household Characteristics): (24939, 17)
Shape of p (Prices): (1007, 3)
Shape of fct (Food Composition Table): (103, 15)
Shape of rdi (Recommended Daily Intake): (20, 14)
